<a href="https://colab.research.google.com/github/chinhegde/Data-Wrangling-Practice/blob/main/Practice_Test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
df = pd.DataFrame({'age': [25, 35, 45] * 10})

In [5]:
df['age'].head()

,age
0,25
1,35
2,45
3,25
4,35


In [9]:
df['age'] = (df['age'].head()*3) + 5

In [10]:
df['age'].head()

,age
0,80.0
1,110.0
2,140.0
3,80.0
4,110.0


In [13]:
# All combinations
# n= 3 -> 1 2 3, k = 2
# output = (1,2), (2,3), (1, 3)

from itertools import combinations

def func(n, k):
  return list(combinations(list(range(1, n+1)), k))

In [14]:
func(3, 2)

[(1, 2), (1, 3), (2, 3)]

In [15]:
func(4, 3)

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

In [38]:
def func1(n, k):

  res = list()

  def create_lists(i, k, curr):

    if k == 0:
      res.append(curr.copy())
      return

    if i > n:
      return

    curr.append(i)
    create_lists(i+1, k-1, curr)
    curr.pop()
    create_lists(i+1, k, curr)

  create_lists(1, k, curr = list())

  print("RES", res)

  return res

In [40]:
func1(4, 3)

RES [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]


[[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]

In [41]:
# user session data - user_id, session_start, session_end, actions

def clean_session_data(data):

  data['session_start'] = pd.to_datetime(data['session_start'])
  data['session_end'] = pd.to_datetime(data['session_end'])

  data['is_valid_session'] = data['session_start'] < data['session_end']

  return data

In [46]:
# viewership data - stream_id, user_id, viewers (count), timestamp
# max number of users per stream over time

def get_max_stream(df):

  res = (
      df.groupby(by='stream_id')['viewers']
      .max()
      .reset_index()
      .rename(columns={'viewers':'max_viewers_per_stream'})
    )

  return res

In [47]:
data = {
    'stream_id': [1, 1, 1, 2, 2, 2],
    'user_id': [101, 102, 103, 201, 202, 203],
    'viewers': [150, 200, 180, 250, 300, 280],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:10:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:05:00',
        '2024-08-10 11:10:00'
    ]
}

data = pd.DataFrame(data)

get_max_stream(data)

,stream_id,max_viewers_per_stream
0,1,200
1,2,300


In [63]:
# User interactions data

data = {
    'user_id': [1, 1, 2, 2, 1, 2, 3, 3],
    'stream_id': [101, 101, 102, 102, 103, 103, 104, 104],
    'action_type': ['view', 'chat', 'view', 'like', 'like', 'chat', 'view', 'view'],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:01:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:06:00',
        '2024-08-10 10:10:00',
        '2024-08-10 10:15:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:01:00'
    ]
}

data = pd.DataFrame(data)

In [70]:
# create a feature matrix
# row -> user_id and count of each action

def feature_engineering(df):

    feature_matrix = pd.pivot_table(df, index='user_id', columns='action_type', aggfunc='size', fill_value=0)
    feature_matrix.reset_index(inplace=True)

    return feature_matrix

In [68]:
feature_engineering(data)

action_type,user_id,chat,like,view
0,1,1,1,1
1,2,1,1,1
2,3,0,0,2


Churn Prediction Feature Engineering:
Question: Suppose you have a dataset of user interactions on Twitch (with columns user_id, stream_id, action_type, timestamp).
You need to build features for a churn prediction model. Write a Python function that creates the following features for each user:
- The total number of unique streams a user interacted with.
- The number of days since the user's last interaction.
- The ratio of views to total interactions per user.

In [82]:
data = {
    'user_id': [1, 1, 2, 2, 1, 2, 3, 3],
    'stream_id': [101, 101, 102, 102, 103, 103, 104, 104],
    'action_type': ['view', 'chat', 'view', 'like', 'like', 'chat', 'view', 'view'],
    'timestamp': [
        '2024-08-10 10:00:00',
        '2024-08-10 10:01:00',
        '2024-08-10 10:05:00',
        '2024-08-10 10:06:00',
        '2024-08-10 10:10:00',
        '2024-08-10 10:15:00',
        '2024-08-10 11:00:00',
        '2024-08-10 11:01:00'
    ]
}

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [112]:
from datetime import datetime

def churn_features(df):

  # streams per user -> count(stream_id)
  # last interaction = datetime.now() - timestamp
  # ratio of views = view count/total actions

  unique_streams = (
      df.groupby('user_id')['stream_id']
      .nunique()
      .reset_index()
      .rename(columns={'stream_id':'unique_streams'})
      )

  total_streams = (
      df.groupby('user_id')['stream_id']
      .count()
      .reset_index()
      .rename(columns={'stream_id':'total_streams'})
      )

  last_interaction = df.groupby('user_id')['timestamp'].max().reset_index()
  last_interaction['days_since_last'] = (datetime.now() - last_interaction['timestamp']).dt.days

  last_interaction.drop(['timestamp'], axis = 1, inplace=True)

  views_per_user = (
      df[df['action_type'] == 'view']
      .groupby('user_id')['stream_id']
      .count()
      .reset_index()
      .rename(columns={'stream_id':'views_count'})
      )

  res = pd.merge(unique_streams, last_interaction, on='user_id')
  res = pd.merge(res, total_streams, on='user_id')
  res = pd.merge(views_per_user, res, on='user_id')

  res['view_to_total_ratio'] = round(res['views_count']/res['total_streams'], 2)
  res.drop(['views_count', 'total_streams'], axis = 1, inplace=True)

  return res

In [113]:
churn_features(df)

,user_id,unique_streams,days_since_last,view_to_total_ratio
0,1,2,3,0.33
1,2,2,3,0.33
2,3,1,3,1.00
